In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/데이터_전처리_파일_선박명.csv', encoding='cp949')

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

대분류, 중분류, 소분류를 feature 값으로 사용하고, target 값으로는 COOLER, GASKET, GEAR, O-RING 등 61개 분류명을 사용하여 다중 분류 모델을 만들 수 있습니다.

전처리 과정에서는 대분류, 중분류, 소분류를 텍스트로 받아와서 토큰화하여 리스트 형태로 만들고, target 값을 숫자로 인코딩하여 넘파이 배열 형태로 만들어줍니다. 그 후, 해당 데이터를 분류 모델에 학습시켜 예측 모델을 만들어서 새로운 입력 데이터에 대해 분류할 수 있습니다.

예를 들어, 다음과 같은 코드로 분류 모델을 만들 수 있습니다. (사용하는 머신러닝 라이브러리와 분류 알고리즘에 따라 코드는 달라질 수 있습니다.)

 CountVectorizer를 사용하여 텍스트 데이터를 feature 벡터로 변환하고, LabelEncoder를 이용하여 target 값을 숫자로 인코딩하였습니다. 이후 train_test_split 함수를 사용하여 데이터를 train, test 데이터로 분리하고, OneVsRestClassifier를 사용하여 SVM 분류기를 만들어 분류 모델을 학습시켰습니다. 마지막으로, 새로운 입력 데이터를 예측하여 target 값을 출력하였습니다.

In [ ]:
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# 텍스트 데이터
data = list(df['Subject'] + ' '+ df['Machinery']  + ' ' + df['Assembly'] + ' ' + df['청구품목'] + ' ' + df['Part No.1'] )

In [ ]:
# target 값
target =list(df['key2'])

In [ ]:
# CountVectorizer를 이용하여 feature 벡터 생성
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data)

In [ ]:
# LabelEncoder를 이용하여 target 값을 숫자로 인코딩
le = LabelEncoder()
y = le.fit_transform(target)

In [ ]:
# train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.metrics import accuracy_score # 정확도 함수

In [ ]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score
def print_metrics(y_test, X_test, model) :
  # 테스트 데이터셋으로 예측
  y_pred = model.predict(X_test)

  # 각종 평가지표 계산
  recall = recall_score(y_test, y_pred, average='macro')
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='macro')
  f1 = f1_score(y_test, y_pred, average='macro')

  print("Recall: {:.4f}".format(recall))
  print("Accuracy: {:.4f}".format(accuracy))
  print("Precision: {:.4f}".format(precision))
  print("F1-score: {:.4f}".format(f1))


In [ ]:
from sklearn.linear_model import LogisticRegression

# Logistic Regression 모델 생성 및 학습
lr = LogisticRegression(random_state=42, C=8, solver="liblinear", multi_class="ovr", max_iter=150)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_acc = accuracy_score(y_test, lr_pred)
print_metrics(y_test, X_test, lr)

Recall: 0.8341
Accuracy: 0.8908
Precision: 0.9083
F1-score: 0.8594


In [ ]:
# 피클로 모델 저장

import pickle 
with open('model.pickle','wb') as fw:
    pickle.dump(lr, fw)

In [21]:
# vectorizer 생성
vectorizer = CountVectorizer()

# feature 벡터 생성
X = vectorizer.fit_transform(data)

# vectorizer 객체 저장
with open('vectorizer.pickle', 'wb') as fw:
    pickle.dump(vectorizer, fw)

In [22]:
# LabelEncoder 생성 및 target 인코딩
le = LabelEncoder()
y = le.fit_transform(target)

# LabelEncoder 객체 저장
with open('label_encoder.pickle', 'wb') as fw:
    pickle.dump(le, fw)

In [ ]:
# 모델 불러오기

import pickle 
with open('model.pickle', 'rb') as f: 
    model = pickle.load(f)